In [54]:
# Cell 1: Data Retrieval

# We import 3 datasets:
# 1. The NQ contract data
# 2. The Nasdaq 100 instruments
# 3. The OHLCV data for the underlying asset - to display chart

import databento as db
import os
import mplfinance as mpf
import numpy as np

api_key = os.environ.get('DATA_BENTO_API_KEY')
# Change the variables below to view performance with different time frames and instruments
# The NQM4 trades between 2023-03-17 and 2024-06-21
# The Nasdaq trades on weekdays from 13:30 to 20:00 UTC
# Note: 5 minutes of data is roughly 5MB
start_time = "2024-06-07T13:30:00"
end_time = "2024-06-07T13:34:00"
# CME Globex market
dataset = 'GLBX.MDP3'
# June E-mini-Nasdaq-100, c.0 is Databento's smart symbology for front month, 
symbols = ['NQ.c.0']
# symbology format must be 'continuous' for the previous contract
symbology_type = "continuous"
# NASDAQ market
underlying_dataset = 'XNAS.ITCH'
underlying_symbols = [
    "MSFT", "AAPL", "NVDA", "AMZN", "META", "AVGO", "GOOGL", "COST", "GOOG", "TSLA",
    "AMD", "NFLX", "PEP", "ADBE", "LIN", "CSCO", "TMUS", "QCOM", "INTU", "INTC",
    "CMCSA", "AMAT", "TXN", "AMGN", "ISRG", "HON", "LRCX", "BKNG", "VRTX", "MU",
    "SBUX", "REGN", "ADP", "MDLZ", "ADI", "KLAC", "GILD", "PANW", "SNPS", "PDD",
    "CDNS", "ASML", "MELI", "CRWD", "CSX", "MAR", "ABNB", "PYPL", "ORLY", "CTAS",
    "PCAR", "MNST", "NXPI", "ROP", "WDAY", "LULU", "MRVL", "ADSK", "CEG", "CPRT",
    "DASH", "FTNT", "DXCM", "ROST", "MCHP", "ODFL", "FAST", "PAYX", "IDXX", "AEP",
    "CHTR", "KHC", "GEHC", "KDP", "MRNA", "CSGP", "AZN", "DDOG", "CTSH", "TTD",
    "EXC", "EA", "FANG", "VRSK", "CDW", "BKR", "CCEP", "ON", "BIIB", "TEAM",
    "ANSS", "ZS", "XEL", "GFS", "DLTR", "MDB", "TTWO", "WBD", "ILMN", "WBA", "SIRI"
    ]
# Schema Docs - https://databento.com/docs/knowledge-base/new-users/fields-by-schema

# Changing the variables below can have breaking changes
# "Top of Book Bid and Offer"
orders_schema = "tbbo"
# Modify the chart_schema and order_frequency together for plotting to work
# "Open High Low Close Volume - 1 second"
chart_schema = "ohlcv-1s"
# Frequency of the orders, "Seconds"
order_frequency = "S"

# Get historical data from Databento
client = db.Historical(api_key)

# Get the data and convert to dataframes
underlying_df = client.timeseries.get_range(
    dataset=underlying_dataset,
    symbols=underlying_symbols,
    schema=orders_schema,
    start=start_time,
    end=end_time,
).to_df()

nq_df = client.timeseries.get_range(
    dataset=dataset,
    stype_in=symbology_type,
    stype_out="instrument_id",
    symbols=symbols,
    schema=orders_schema,
    start=start_time,
    end=end_time,
).to_df()

nq_ohlc_df = client.timeseries.get_range(
    dataset=dataset,
    schema=chart_schema,
    stype_in=symbology_type,
    symbols=symbols,
    start=start_time,
    end=end_time,
).to_df()

2024-06-11 21:17:15,784 - INFO     - Initialized Historical(gateway=https://hist.databento.com)


In [55]:
# Cell 2: Trade logic

from trade_utils import prepare_dataframes, group_and_aggregate, create_trades_df, modify_trades, calculate_pnl

# Add NQ bid / ask to all orders
nq_and_underlying_df = prepare_dataframes(nq_df, underlying_df)
# Group trade data in 1 second intervals
grouped_df = group_and_aggregate(nq_and_underlying_df, order_frequency)
# Create trades dataframe and add trades when conditions are met
trades_df = create_trades_df(grouped_df, nq_ohlc_df)
# Modify the trade dataframe to restrict to 1 contract open at a time
modify_trades(trades_df)
# Add a pnl and total_pnl column to the trades dataframe
calculate_pnl(trades_df)


In [51]:
# Cell 3: Visualize dataframes - this cell is optional and primarily used for debugging

import dtale

# Visualize various dataframes for analysis
# grouped_df
# nq_and_underlying_df
# trades_df
dtale.show(trades_df)

In [57]:
# Cell 4: Plot trades

from chart_utils import format_chart_title, create_trade_scatter, create_pnl_plot
# Take time and instrument parameters from Cell 1 to label the chart
chart_title = format_chart_title(symbols, start_time, chart_schema)
# Create separete scatter plots for buy and sell trades
# "B" = Buy, "S" = Sell, 'g' = green, 'r' = red, '^', 'v' = scatter plot symbol
buy_plot = create_trade_scatter(trades_df, 'B', 'g', '^')
sell_plot = create_trade_scatter(trades_df, 'S', 'r', 'v')
# Create a PNL line plot
pnl_plot = create_pnl_plot(trades_df)

mpf.plot(
    nq_ohlc_df,
    type="candle",
    title=chart_title,
    ylabel="Price",
    xlabel="Time UTC",
    addplot=[buy_plot, sell_plot, pnl_plot],
)

